In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import houses as ho
import numpy as np

In [2]:
df = pd.read_csv('datasets/house_train_raw.csv')

In [3]:
df, numerical = ho.transform(df)

In [4]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,noApply,Reg,Lvl,AllPub,...,0,noApply,noApply,noApply,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,noApply,Reg,Lvl,AllPub,...,0,noApply,noApply,noApply,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,noApply,IR1,Lvl,AllPub,...,0,noApply,noApply,noApply,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,noApply,IR1,Lvl,AllPub,...,0,noApply,noApply,noApply,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,noApply,IR1,Lvl,AllPub,...,0,noApply,noApply,noApply,0,12,2008,WD,Normal,250000


In [5]:
# get the locations
X = df.loc[:, numerical]
y = df.loc[:, 'SalePrice']
X['constant'] = 1

# get x_train nad y_train
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.4, random_state=0)

# get x, y val, and x, y test
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.5, random_state=0)

In [6]:
x_t = np.array(X_train)
y_t = np.array(y_train)

In [7]:
theta = ho.normalEqn(x_t, y_t)

In [8]:
theta

array([ 2.52455834e-11,  1.48851586e-06,  3.99482815e-07, -1.58561875e-09,
        5.77751052e-07, -2.24019914e-10, -5.69979619e-11,  2.20765628e-11,
        8.46193871e-09,  1.51239021e-11, -9.20632459e-11, -4.94608798e-11,
        1.00000000e+00, -9.93313733e-08])

In [9]:
predict = np.matmul(X_val, theta)

In [11]:
ho.rmsle(predict, y_val)

7.494573987278772e-12